In [3]:
using DataFrames
using CSV
using CPLEX
include("../instances_io.jl")
include("PL.jl");
include("../Heur/Heur.jl")

L = [3,4,5,6,7,8] # number of lines in the grid-like graph
C = [3,4,5,6,7,8] # number of columns in the grid-like graph
K = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20] # number of edges the adversary can penalize
MAXC = 5 # range of initial edge cost
MAXD = [0,1,2,3,4,5,6,7,8,9,10,20,30,40,50,75,100] # range of penalized edge cost
l = 5 # number of lines in the grid-like graph
c = 5 # number of columns in the grid-like graph
k = 10 # number of edges the adversary can penalize
maxc = 5 # range of initial edge cost
maxd = 50 # range of penalized edge cost

df = DataFrame(PL=Int[], Param = String[], Param_value = Int[], Iteration=Int[], Time=Float64[], Cuts=Int[])
for par in [L,C,K,MAXD]
    l = 5 # number of lines in the grid-like graph
    c = 5 # number of columns in the grid-like graph
    k = 10 # number of edges the adversary can penalize
    maxc = 5 # range of initial edge cost
    maxd = 50 # range of penalized edge cost
    for par_value in par
        for it in 1:1
            if par==L
                sv = generate(par_value, c, k, maxc, maxd)
                name="L"
            elseif par==C
                sv = generate(l, par_value, k, maxc, maxd)
                name="C"
            elseif par==K
                sv = generate(l, c, par_value, maxc, maxd)
                name="K"
            else
                sv = generate(l, c, k, maxc, par_value)
                name="MAXD"
            end
            timer = @elapsed obj, x_opt = solve_PL1(sv, CplexSolver(CPX_PARAM_MIPDISPLAY=0))
            push!(df, (1, name, par_value, it, timer, 0))
            timer = @elapsed obj, x_opt, count = solve_PL2(sv, CplexSolver(CPX_PARAM_MIPDISPLAY=0))
            push!(df, (2, name, par_value, it, timer, count))
            timer = @elapsed obj, x_opt, count = solve_PL3(sv, CplexSolver(CPX_PARAM_MIPDISPLAY=0))
            push!(df, (3, name, par_value, it, timer, count))
            timer = @elapsed obj, x_opt = tree_search(sv, 100)
            push!(df, (4, name, par_value, it, timer, 0))
            CSV.write("Results.csv", df)
        end
    end
end

        

UndefVarError: UndefVarError: CbcSolver not defined